# ⛰️ Uçurum Yürüyüşü
---

Bu meydan okumada, pekiştirmeli öğrenme modellerini eğitmek için Gymnasium ve Stable Baselines3 kullanımının temel yönlerini öğreneceksiniz. Bu meydan okuma, ortamları kurma, modelleri eğitme, performanslarını görselleştirme ve son olarak eğitilmiş bir modeli bir ortamla etkileşim kurmak için kullanma konularında bütüncül bir anlayış kazandırmanız için tasarlanmıştır.

---
Tüm import ifadelerinizi notebook'unuzun en üstüne koyarak notebook'unuzu tekrar çalıştırmayı kolaylaştırmak iyi bir alışkanlıktır.

Meydan okumalar boyunca import ifadeleri eklemeniz gerekecek. Hepsini aşağıdaki hücreye ekleyin.

In [2]:
import os
import time
import numpy as np

import gymnasium as gym

from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv

---

### 🎯 Meydan Okumanın Hedefleri

#### 🏋️‍♀️ Gymnasium'a Aşinalık Kazanın:
Gymnasium'dan [CliffWalking ortamını](https://gymnasium.farama.org/environments/toy_text/cliff_walking/) keşfederek başlayacağız. Ortamları yönetmek ve görselleştirmek için kullanılan temel yöntemleri anlayın. Ortamları yüklemeyi ve sıfırlamayı, eylemler almayı ve sonuçları görselleştirmeyi öğrenin.

#### 🤖 Stable Baselines3 Kullanmayı Öğrenin:
Bir pekiştirmeli öğrenme modelini kurmak ve eğitmek için Stable Baselines3 kütüphanesini kullanın. Bir modeli yapılandırmayı, eğitim parametrelerini ayarlamayı ve Deep Q-Network (DQN) gibi popüler bir algoritma kullanarak eğitim sürecini başlatmayı öğreneceksiniz.

#### 📈 Eğitim Performansını Görselleştirin:
Modelinizin eğitim performansını izlemek ve çizmek için günlük tutma ve TensorBoard gibi görselleştirme araçlarını uygulayın. Eğitim stratejilerinizin etkinliğini değerlendirmek için bölüm başına ödüller ve öğrenme eğrileri gibi metrikleri analiz edin.

#### 💾 Eğitilmiş Bir Modeli Yükleyin ve Dağıtın:
Eğitim sonrasında, eğitilmiş bir modeli kaydetmeyi ve daha sonra yüklemeyi öğrenin. Bu modeli, ajanın CliffWalking ortamıyla etkileşim kurduğu bir simülasyon çalıştırmak için kullanın ve öğrenilen politikaları ortamda etkili bir şekilde gezinmek için uygulayın.

#### 🔍 Değerlendirin ve Düşünün:
Eğitilmiş modelin ortam içindeki gerçek zamanlı etkileşimlerdeki performansını değerlendirin. Öğrenme süreci ve ajanın davranışı üzerine düşünün, farklı yapılandırmaların ve eğitim sürelerinin sonuçları nasıl etkileyebileceğini anlayın.

---
### Bölüm 1: CliffWalking Ortamını Keşfetmek

Bu ilk görevde, Gymnasium'dan [CliffWalking ortamını](https://gymnasium.farama.org/environments/toy_text/cliff_walking/) kuracaksınız.

#### 📝 Takip Edilecek Adımlar:

#### 0. ⚙️ Paketi içe aktarın:
Notebook'unuzun en üstündeki hücreye gymnasium'u içe aktarın. Kodumuzda `gym` olarak kullanabildiğimizden emin olun.

#### 1. 🗂️ Ortamı Yükleyin:
- `CliffWalking` ortamını yüklemek için `.make()` metodunu kullanın.  
- Ortamı görselleştirmek için `render_mode`'u uygun şekilde ayarlayın.

Not: `gymnasium` dokümantasyonu size `CliffWalking-v1` kullanmanızı söylüyor, ancak bu henüz uyumluluk nedenleriyle kullandığımız `gymnasium` sürüm 1.0.0'da bulunmuyor. Bunun yerine `CliffWalking-v0` kullanın.

In [4]:
import gymnasium as gym

# CliffWalking var mı?
print([k for k in gym.registry.keys() if "CliffWalking" in k])

['CliffWalking-v1', 'CliffWalkingSlippery-v1', 'tabular/CliffWalking-v0']


In [5]:
CLIFF_ID = "CliffWalking-v1"   # biz bunu kullanacağız

#### 2. 🔄 Ortamı Başlatın:
Ortamı başlangıç durumuna getirmek için `.reset()` metodunu çağırın — onunla etkileşim kurmadan önce gereklidir. `.reset()` metodu bir durum döndürür: başlangıç durumunu. Bunu bir değişkende saklayın ve yazdırın.

> `UserWarning: pkg_resources is deprecated ...` uyarısını görebilirsiniz. Bunu görmezden gelebilirsiniz.

In [6]:
env = gym.make(CLIFF_ID, render_mode="human")  # pencere açar
obs, info = env.reset()
print("obs:", obs, "info:", info)

obs: 36 info: {'prob': 1}


`(36, {'prob': 1})` çıktısı ortamın durumu hakkında iki bilgi parçası sağlar:

**🔢 Durum İndeksi (36)**: Bu sayı, ajanın başlangıçta ortam ızgarası içindeki belirli konumunu temsil eder. Örneğin 'CliffWalking-v0' ortamında, `36` indeksi ajanın bölümü başlattığı başlangıç durumuna karşılık gelir.

**🎲 Olasılık Bilgisi ({'prob': 1})**: Bu sözlük, durum hakkında, özellikle geçiş olasılığı hakkında ek ayrıntılar gösterir. `'prob'` anahtarının `1` değeri, bu duruma geçişin 1 olasılığıyla gerçekleştiğini, yani kesin olduğunu gösterir. Bu mantıklıdır çünkü ajan mutlaka bu durumda başlayacaktır.

#### 3. 👀 Ortamı Görselleştirin:
Herhangi bir eylem almadan önce ortamı görüntülemek ve düzenini anlamak için `.render()` metodunu kullanın.

Kurulumunuza bağlı olarak, yeni durum önceki adımdan sonra zaten işlenmiş olabileceği için bu adıma ihtiyacınız olmayabilir.

#### 4. 🧹 Ortamı Kapatın:
İşiniz bittiğinde kaynakları serbest bırakmak için `.close()` çağırarak ortamı düzgün bir şekilde kapatın.

In [7]:
env.render()

action = env.action_space.sample()
print("action:", action)

obs, reward, terminated, truncated, info = env.step(action)
done = terminated or truncated

print("obs:", obs, "reward:", reward, "done:", done, "info:", info)

env.render()
env.close()

action: 0
obs: 24 reward: -1 done: False info: {'prob': 1.0}


<details>
  <summary markdown='span'>⚠️ <strong>macOS için kapatma hakkında not</strong></summary>
  
  `.close()` ortam penceresini kapatmayacak. Bu sorun değil, **açık bırakabilirsiniz**. Pencere sonraki adımlarda tekrar kullanılacak.

  Meydan okumanın sonunda ve **sadece sonunda**, ortam penceresini kapatmak için onu kendiniz zorla sonlandırmanız gerekecek:
  1. Pencereyi bulun ve kapatmak için kırmızı düğmeye tıklayın. (Bu başarısız olacak.)
  1. Ekranınızın sol üst köşesinde Apple sembolüne tıklayın.
  1. `Force Quit`'i (Zorla Çıkış) seçin.
  1. İşlem listesinde `python (Not Responding)` işlemini bulun. Onu seçin.
  1. `Force Quit` düğmesine tıklayın.

  Bu çekirdeğinizi sonlandıracağından, **bunu sadece meydan okumanın sonunda yapın**.

  Bu durum Jupyter Notebook içinde çalıştırdığımız için oluyor. Kodunuzu bir `.py` dosyasına taşırsanız, bu davranışı görmezsiniz.

</details>

---
### Bölüm 2: Ortamla Etkileşim Kurmak

Şimdi ortamı tekrar yükleyelim, rastgele bir adım atalım ve sonucu gösterelim. Bu, ortamlarla etkileşim kurmak için temel metotları tanıtacak — ajanların nasıl hareket ettiğini ve geri bildirim aldığını anlamanıza yardımcı olacak.

#### 1. 🗂️ Ortamı Yükleyin
- Ortamı yükleyin.
- Başlangıç durumuna sıfırlayın.
- Başlangıç durumunu yazdırın.

In [8]:
import gymnasium as gym

# Ortamı yükle (görsel pencere istiyorsan "human", istemiyorsan None yap)
env = gym.make("CliffWalking-v1", render_mode="human")

# Başlangıç durumuna sıfırla
obs, info = env.reset()

# Başlangıç durumunu yazdır
print("obs:", obs, "info:", info)

obs: 36 info: {'prob': 1}


#### 2. 🎲 Bir Eylem Örnekleyin:
- Ortamın eylem alanından rastgele bir eylem seçmek için `.action_space.sample()` kullanın — keşfi simüle eder.
- Bunu bir değişkende kaydedin.
- Eylemin türünü ve değerini inceleyin.
- Hücreyi birkaç kez çalıştırın. Hangi değerleri görüyorsunuz? Bunlar neyi temsil ediyor?

In [9]:
# Rastgele bir eylem seç
action = env.action_space.sample()

print("action type:", type(action))
print("action value:", action)

action type: <class 'numpy.int64'>
action value: 2


#### 2. 🦶 Eylemi Gerçekleştirin:
- `.step()` kullanarak eylemi uygulayın.  
- Bu şunları döndürür:  
  - Yeni durum  
  - Ödül  
  - Bir `done` bayrağı (bölüm bitmiş mi değil mi)  
  - Ek bilgi (varsa)
- "tuple unpacking" kullanarak bu dönen değerlerin her birini bir değişkende kaydedin. Onları yazdırın. Yeni durum, ödül ve done dönüş değerlerini anlıyor musunuz?
- Bu **sıfırla > eylem > adım** dizisini birkaç kez çalıştırın ve farklı sonuçları kontrol edin.

<details>
  <summary markdown='span'>
  💡 Tuple unpacking?
  </summary>

  Bir fonksiyon tuple döndürürse, tuple'ın farklı elemanlarını hemen farklı değişkenlere kaydedebilirsiniz.

  Örnek:

  Şu fonksiyonunuz olduğunu düşünün:

  ```python
  def surface_and_circumference(a, b):
    """Uzunluğu `a` ve genişliği `b` olan dikdörtgenin 
    alanını ve çevresini döndürür."""
    return a*b, 2*a + 2*b
  ```

  Bunun yerine:

  ```python
  result = surface_and_circumference(4, 2)
  surface = result[0]
  circumference = result[1]
  ```

  Hemen şunu yapabilirsiniz:

  ```python
  surface, circumference = surface_and_circumference(4, 2)
  ```

  Kodunuzun geri kalanında sadece surface kullanacaksanız, diğer dönüş değerleri için `_` kullanmak yaygın bir alışkanlıktır. Bu, diğer programcılara kalan değerleri attığınızın bir işaretidir.

  Örnek:
  
  ```python
  surface, _ = surface_and_circumference(4, 2)
  # Buradan sonra sadece surface'a ihtiyaç duyan kod gelir
  ```

</details>

In [10]:
# Eylemi uygula
obs, reward, terminated, truncated, info = env.step(action)
done = terminated or truncated

print("obs:", obs, "reward:", reward, "done:", done, "info:", info)

obs: 36 reward: -1 done: False info: {'prob': 1.0}


**📍 Yeni Durum**: Bu sayı, belirtilen eylemi gerçekleştirdikten sonra ajanın durumunu temsil eder. `CliffWalking` ortamında durum, ajanın hareket ettiği ortam ızgarasındaki belirli bir konuma karşılık gelir.

**💸 Ödül**: Ödül değeri, ajanın eyleminin sonucu olarak ortam tarafından verilen anlık geri bildirimi gösterir. Birçok ızgara tabanlı ortamda, bunun gibi negatif bir ödül genellikle bir cezayı ifade eder ve alınan eylemin optimal olmayabileceğini veya diğer stratejileri teşvik etmek için cezalandırıldığını gösterir.

**🚦 Done**: Boolean değer bölümün bitip bitmediğini gösterir. Bu durumda, `False` bölümün hala devam ettiği ve ajanın bölümü sona erdirecek terminal bir duruma (hedef veya çukur gibi) ulaşmadığı anlamına gelir.

#### 3. 👁️ Sonucu Görselleştirin:
- Eylem sonrasında ortamı görselleştirmek için tekrar `.render()` çağırın — ve durumun nasıl değiştiğini görün.
- Render penceresi görmüyorsanız, muhtemelen diğer pencerelerinizin arkasında gizli veya başka bir masaüstünde gösteriliyor.

In [11]:
env.render()

#### 4. 🧹 Ortamı Kapatın:
İşiniz bittiğinde kaynakları serbest bırakmak için `.close()` çağırarak ortamı düzgün bir şekilde kapatın.

In [12]:
# 4) Ortamı kapat
env.close()
print("Environment closed ✅")

Environment closed ✅


---
### Bölüm 3: Ortamla Yönlendirilmiş Etkileşim

Bu sefer rastgele bir eylem seçmek yerine kasıtlı bir adım atacaksınız — **YUKARI** hareket edeceğiz. Bu, pekiştirmeli öğrenmede amaçlı karar verme fikrini pekiştirir.

#### 1. 🚀 Başlatın ve İlk Durumu Yazdırın:
- Ortamı yükleyin ve başlangıç durumunu almak için `.reset()` çağırın.  
- Herhangi bir eylem almadan önce başlangıç durumunu görüntülemek için `print()` kullanın.
- Şu çıktıyı almalısınız `(36, {'prob': 1})`

In [13]:
import gymnasium as gym

# 1) Ortamı yükle
env = gym.make("CliffWalking-v1", render_mode="human")

# 2) Başlangıç durumuna sıfırla
obs, info = env.reset()

# 3) Başlangıç durumunu yazdır
print("initial obs:", obs, "info:", info)

initial obs: 36 info: {'prob': 1}


#### 2. ⬆️ 'YUKARI' Eylemini Belirleyin ve Gerçekleştirin

- `.action_space`'i ve dokümantasyonu kontrol ederek **'YUKARI'** eylemi için indeksi bulun.  
- Bu eylemi gerçekleştirmek için `.step(action_index)` kullanın.  
- Şunları görmek için sonucu yazdırın:
  - Yeni durum  
  - Alınan ödül  
  - Bölümün bitip bitmediği (`done` bayrağı)

In [14]:
# Aksiyon uzayını incele
print("action_space:", env.action_space)

# ToyText ortamlarında genelde şu mapping vardır:
# 0=UP, 1=RIGHT, 2=DOWN, 3=LEFT
UP = 0

# UP eylemini uygula
new_obs, reward, terminated, truncated, info = env.step(UP)
done = terminated or truncated

print("UP action index:", UP)
print("new obs:", new_obs)
print("reward:", reward)
print("done:", done)
print("info:", info)

action_space: Discrete(4)
UP action index: 0
new obs: 24
reward: -1
done: False
info: {'prob': 1.0}


#### 3. 🖼️ Ortamı Görselleştirin ve Kapatın

- Eylem sonrasında ortamı görselleştirmek için `.render()` kullanın — durumun nasıl değiştiğini görün.
- Ardından ortamı düzgün bir şekilde kapatmak ve sistem kaynaklarını serbest bırakmak için `.close()` çağırın.

In [15]:
# Ortamı görselleştir (bazı kurulumlarda zaten pencere güncellenmiş olur)
env.render()

# Ortamı kapat
env.close()
print("Environment closed ✅")

Environment closed ✅


---
### Bölüm 4: Bir RL Algoritması Eğitmek

İlk olarak, `CliffWalking-v0` ortamını Stable Baselines3 modelleriyle eğitim için hazırlayacaksınız. Bu, RL algoritmalarıyla uyumluluğu sağlamak için uygun kurulum ve sarmalama içerir.

#### 🧱 1. Ortamı Yükleyin:
- `CliffWalking-v0` ortamını oluşturmak için `gym.make()` kullanın.  
- `render_mode='human'` ayarlayın → Etkileşim sırasında görsel geri bildirim sağlar.

In [16]:
import os
import gymnasium as gym

from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv

In [17]:
# 1) Ortamı oluştur (izlemek için human)
env = gym.make("CliffWalking-v1", render_mode="human")

print("Obs space:", env.observation_space)
print("Action space:", env.action_space)

Obs space: Discrete(48)
Action space: Discrete(4)


#### 🧵 2. Ortamı Sarın

Ortamınızı [`DummyVecEnv`](https://stable-baselines3.readthedocs.io/en/v1.0/guide/vec_envs.html#dummyvecenv) ile sarmak küçük ama kritik bir adımdır — RL kurulumunuzun Stable Baselines3 ile sorunsuz çalışmasını sağlar.

Devam edin ve ortamı Stable Baselines3'ten [`DummyVecEnv`](https://stable-baselines3.readthedocs.io/en/v1.0/guide/vec_envs.html#dummyvecenv) kullanarak sarın.  👇

In [18]:
# 2) SB3 uyumluluğu için sarmala (tek ortam bile olsa)
env = DummyVecEnv([lambda: env])

print("Wrapped env:", env)

Wrapped env: <stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x30a8a5e80>


#### 🧵 `DummyVecEnv` Hakkında Daha Fazla

`DummyVecEnv`, Gym ortamlarını vektörleştirerek RL modelleriyle uyumlu hale getiren Stable Baselines3'ten bir sarmalayıcıdır.

#### ⚙️ Ne Yapar?

- **📦 API'yi Standartlaştırır**  
  Ortamın Stable Baselines3 algoritmaları için beklenen formatla eşleşmesini sağlar.

- **📊 Toplu İşlemeyi Etkinleştirir**  
  Tek bir ortamla bile etkileşimler bir toplu gibi işlenir — daha sonra `SubprocVecEnv` gibi araçlarla ölçekleme için gereklidir.

- **🔗 Uyumluluğu Sağlar**  
  `reset()` ve `step()`'i eğitim döngüleri içinde doğru çalışacak şekilde sarar.

#### 3. SB3'den Bir DQN Modeli Başlatın

Artık ortamınız hazır olduğuna göre, modeli başlatmanız gerekir. Bu durumda, Stable Baselines3 kullanarak bir Deep Q-Network (DQN) yapılandıracaksınız. DQN, Q-değerlerini tahmin etmek için bir sinir ağı kullanır — belirli bir durumda her eylem için beklenen ödülü tahmin eder.

#### 📝 Takip Edilecek Adımlar:
- `stable_baselines3`'den `DQN`'i içe aktarın. Bunu notebook'un en üstündeki hücreye ekleyin.
- Aşağıdaki parametrelerle DQN modelinin bir örneğini başlatın:
    - `'MlpPolicy'` kullanın — gözlemleri eylemlere eşleyen temel bir sinir ağı.  
    - `env`'i ortamınıza ve ayrıntılı eğitim günlükleri için `verbose=1`'e ayarlayın.
    - Eğitim metriklerini izlemek için `tensorboard_log` parametresi ekleyin. Bunu daha sonra TensorBoard ile eğitimi takip etmek için kullanacağız.

In [19]:
# TensorBoard log klasörü
log_dir = "./dqn_cliff_tensorboard"
os.makedirs(log_dir, exist_ok=True)

# 3) DQN modelini başlat
model = DQN(
    policy="MlpPolicy",
    env=env,
    verbose=1,
    tensorboard_log=log_dir,
)

print("DQN model initialized ✅")

Using cpu device
DQN model initialized ✅


### 4. 🏋️‍♂️ DQN Modelini Eğitin ve Kaydedin

DQN modelinizi eğitme ve gelecekte kullanmak üzere kaydetme zamanı 🥳

#### 📝 Takip Edilecek Adımlar:

#### ⏱️ Eğitim Adım Sayısını Ayarlayın
- Modeli ne kadar süre eğiteceğinizi (ortamla etkileşim sayısı cinsinden) tanımlayın.  
- Şimdilik şunu kullanın: `total_timesteps = 1000`

#### 🧠 Modeli Eğitin
- Eğitimi başlatmak için DQN modelinizde `.learn()` çağırın.  
- Model, geri bildirimlere dayanarak zaman içinde politikasını geliştirecek.

#### 💾 Eğitilmiş Modeli Kaydedin
- Modeli diske kaydetmek için `.save()` kullanın.  
- Doğru kaydedildiğini doğrulamak için dosya yolunu yazdırın.

🎥 Eğitim sırasında işlenmiş ortamdaki adımları takip edebilirsiniz.

In [20]:
# 4) Eğitim adım sayısı
total_timesteps = 1000

# Modeli eğit (progress_bar=True destekleniyorsa bar gösterir)
model.learn(total_timesteps=total_timesteps, progress_bar=True)

# Kaydetme yolu
model_path = "dqn_cliffwalking_1k"

# Modeli kaydet
model.save(model_path)

print(f"Model saved ✅ -> {model_path}.zip")

Logging to ./dqn_cliff_tensorboard/DQN_1


Output()

Model saved ✅ -> dqn_cliffwalking_1k.zip


In [21]:
# Eğitim sonrası ortamı kapat
env.close()
print("Env closed ✅")

Env closed ✅


Tebrikler, ilk RL modelinizi eğittiniz!

---

### Bölüm 5: Görselleştirme Olmadan Verimli Eğitim

Bu bölümde, ortamı hazırlama, modeli yükleme ve eğitme sürecinin tamamından geçeceksiniz, ancak görsel işleme olmadan. Eğitim aşamasında görselleştirmeyi devre dışı bırakmak, hesaplama yükünü azalttığı için eğitim sürecini önemli ölçüde hızlandırabilir.

#### 📝 Takip Edilecek Adımlar:

#### 🧱 1. Ortamı Yükleyin ve Hazırlayın
- Görsel çıktıyı devre dışı bırakmak için `render_mode=None` kullanın.

#### ⚙️ 2. DQN Modelini Yapılandırın ve Başlatın
- Daha önce olduğu gibi `'MlpPolicy'` ve `DummyVecEnv` kullanın.  
- Günlük çıktısı istiyorsanız `verbose=1`'i koruyun.
- TensorBoard ile eğitimi takip edebilmemiz için bir TensorBoard günlükleme konumu `./dqn_tensorboard` ekleyeceğiz.

#### ⏱️ 3. Eğitim Parametrelerini Ayarlayın ve Eğitin
- Daha iyi öğrenme için adım sayısını artırın (örn. `total_timesteps = 100_000`).  
- Eğitimi başlatmak için `.learn()` çağırın.

#### 💾 4. Eğitilmiş Modeli Kaydedin
- Modelinizi saklamak için `.save("dqn_cliffwalking_fast")` kullanın.

Bu adımları izleyerek, ortamı görselleştirmenin ek hesaplama yükü olmadan verimli bir şekilde bir DQN modeli eğitir ve kaydedersiniz. Bu yaklaşım özellikle karmaşık modeller eğitirken veya sınırlı hesaplama kaynakları kullanırken faydalıdır.

In [22]:
import os
import gymnasium as gym

from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv

In [23]:
# 1) Görsel yok: daha hızlı eğitim
env_fast = gym.make("CliffWalking-v1", render_mode=None)

# 2) SB3 uyumluluğu için sar
env_fast = DummyVecEnv([lambda: env_fast])

print("Fast env ready ✅")

Fast env ready ✅


In [24]:
log_dir_fast = "./dqn_cliff_tensorboard_fast"
os.makedirs(log_dir_fast, exist_ok=True)

model_fast = DQN(
    policy="MlpPolicy",
    env=env_fast,
    verbose=1,
    tensorboard_log=log_dir_fast,
)

print("DQN fast model initialized ✅")

Using cpu device
DQN fast model initialized ✅


In [25]:
log_dir_fast = "./dqn_cliff_tensorboard_fast"
os.makedirs(log_dir_fast, exist_ok=True)

model_fast = DQN.load("dqn_cliffwalking_1k", env=env_fast, print_system_info=True)
model_fast.tensorboard_log = log_dir_fast  # logları buraya yazsın

print("Loaded dqn_cliffwalking_1k and ready to continue ✅")

== CURRENT SYSTEM INFO ==
- OS: macOS-26.2-arm64-arm-64bit Darwin Kernel Version 25.2.0: Tue Nov 18 21:09:56 PST 2025; root:xnu-12377.61.12~1/RELEASE_ARM64_T6041
- Python: 3.12.9
- Stable-Baselines3: 2.7.1
- PyTorch: 2.9.1
- GPU Enabled: False
- Numpy: 1.26.4
- Cloudpickle: 3.1.2
- Gymnasium: 1.2.3

== SAVED MODEL SYSTEM INFO ==
- OS: macOS-26.2-arm64-arm-64bit Darwin Kernel Version 25.2.0: Tue Nov 18 21:09:56 PST 2025; root:xnu-12377.61.12~1/RELEASE_ARM64_T6041
- Python: 3.12.9
- Stable-Baselines3: 2.7.1
- PyTorch: 2.9.1
- GPU Enabled: False
- Numpy: 1.26.4
- Cloudpickle: 3.1.2
- Gymnasium: 1.2.3

Loaded dqn_cliffwalking_1k and ready to continue ✅


In [26]:
total_timesteps = 100_000

model_fast.learn(total_timesteps=total_timesteps, progress_bar=True)

model_fast_path = "dqn_cliffwalking_fast"
model_fast.save(model_fast_path)

print(f"Model saved ✅ -> {model_fast_path}.zip")

Logging to ./dqn_cliff_tensorboard_fast/DQN_1


Output()

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 4        |
|    fps              | 4551     |
|    time_elapsed     | 11       |
|    total_timesteps  | 54454    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 5.79e-05 |
|    n_updates        | 13813    |
----------------------------------


----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 8        |
|    fps              | 4546     |
|    time_elapsed     | 12       |
|    total_timesteps  | 57584    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.000406 |
|    n_updates        | 14595    |
----------------------------------


----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 12       |
|    fps              | 4513     |
|    time_elapsed     | 16       |
|    total_timesteps  | 73020    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 6.43e-05 |
|    n_updates        | 18454    |
----------------------------------


...

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 44       |
|    fps              | 4470     |
|    time_elapsed     | 20       |
|    total_timesteps  | 93351    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.000203 |
|    n_updates        | 23537    |
----------------------------------


----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 48       |
|    fps              | 4463     |
|    time_elapsed     | 21       |
|    total_timesteps  | 95833    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.000535 |
|    n_updates        | 24158    |
----------------------------------


----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 52       |
|    fps              | 4456     |
|    time_elapsed     | 21       |
|    total_timesteps  | 97752    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 2.64e-05 |
|    n_updates        | 24637    |
----------------------------------


----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 56       |
|    fps              | 4448     |
|    time_elapsed     | 22       |
|    total_timesteps  | 99316    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.006    |
|    n_updates        | 25028    |
----------------------------------


Model saved ✅ -> dqn_cliffwalking_fast.zip


In [27]:
env_fast.close()
print("Fast env closed ✅")

Fast env closed ✅


#### 📊 Eğitim Günlük Metriklerini Anlamak

Bir pekiştirmeli öğrenme modelini eğitirken, çeşitli metrikler öğrenme ilerlemesini ve performansını izlemeye yardımcı olur.

#### 🎲 Rollout Metrikleri

- **exploration_rate** → Rastgele bir eylem alma olasılığı.  
  - Yüksek = daha fazla keşif  
  - Düşük = daha fazla istismar  

#### ⏱️ Zaman ile İlgili Metrikler

- **episodes** → Tamamlanan bölüm sayısı.  
- **fps** → Saniye başına kare (eğitimin ne kadar hızlı çalıştığı).  
- **time_elapsed** → Eğitim başladığından beri geçen toplam süre (saniye).  
- **total_timesteps** → Ortamda atılan toplam adım sayısı.

#### 🧠 Eğitim Metrikleri

- **learning_rate** → Model ağırlıklarına yapılan güncellemelerin boyutu.  
  - Düşük = daha yavaş ama daha kararlı öğrenme  
- **loss** → Modelin tahmin hatası.  
  - Azalan kayıp = öğrenme çalışıyor  
- **n_updates** → Modelin ağırlıklarını kaç kez güncellediği.

#### 🔍 Nasıl Yorumlanır

- **⬇️ exploration_rate** → Ajan keşiften öğrenilmiş davranışa geçiyor.  
- **⚡ Yüksek fps** → Eğitim verimli çalışıyor.  
- **📉 Azalan kayıp** → Model gelişiyor ve daha az hata yapıyor.

#### 🖥️ TensorBoard'u Başlatın

TensorBoard eğitimin nasıl gittiğini takip etmenizi sağlar. Etkileşimli pano öğrenme eğrilerini, ödül trendlerini ve daha fazlasını gösterir. 

TensorBoard kullanmak için:

1. Bir terminal penceresi açın.
1. Meydan okuma klasöründe olduğunuzdan emin olun!
1. Bu komutu çalıştırın (eğitim sırasında ayarladığınız yol ile değiştirmeniz gerekebilir):
   ```bash
   tensorboard --logdir=./dqn_cliff_tensorboard/
   ```
1. Terminalde gösterilecek bağlantıyı takip edin (muhtemelen `localhost:6006`).

Modeliniz hala eğitilirken TensorBoard'u başlatabilirsiniz: amacı eğitimin nasıl gittiğini takip etmektir. Eğitimin başlangıcında `No dashboards are active for the current data set.` uyarısı alabilirsiniz. **Biraz sabırlı olun: ilk bölüm bitene kadar hiçbir şey görmeyeceksiniz.**

TensorBoard'u notebook'unuzun içinde açmak da mümkün:

```bash
%load_ext tensorboard
%tensorboard --logdir=./dqn_cliff_tensorboard/
```

### 📦 Bölüm 6: Eğitilmiş Bir Model Kullanın

Sıfırdan tekrar eğitmek yerine, ajanınızın nasıl davrandığını hızlıca gözlemlemek için az önce eğittiğiniz modeli yükleyebilirsiniz.  
Alternatif olarak, mevcut olduğu takdirde daha güçlü önceden eğitilmiş bir model yükleyebilirsiniz.

#### 📥 1. Önceden Eğitilmiş Modeli Yükleyin

- Modeli diskten yüklemek için `.load("önceden_eğitilmiş_modelinizin_yolu")` kullanın.  
- Yolu kaydettiğiniz modelin gerçek konumuyla değiştirin.

In [28]:
import gymnasium as gym
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv

In [29]:
model_path = "dqn_cliffwalking_fast"
model = DQN.load(model_path, print_system_info=True)
print("Model loaded ✅")

== CURRENT SYSTEM INFO ==
- OS: macOS-26.2-arm64-arm-64bit Darwin Kernel Version 25.2.0: Tue Nov 18 21:09:56 PST 2025; root:xnu-12377.61.12~1/RELEASE_ARM64_T6041
- Python: 3.12.9
- Stable-Baselines3: 2.7.1
- PyTorch: 2.9.1
- GPU Enabled: False
- Numpy: 1.26.4
- Cloudpickle: 3.1.2
- Gymnasium: 1.2.3

== SAVED MODEL SYSTEM INFO ==
- OS: macOS-26.2-arm64-arm-64bit Darwin Kernel Version 25.2.0: Tue Nov 18 21:09:56 PST 2025; root:xnu-12377.61.12~1/RELEASE_ARM64_T6041
- Python: 3.12.9
- Stable-Baselines3: 2.7.1
- PyTorch: 2.9.1
- GPU Enabled: False
- Numpy: 1.26.4
- Cloudpickle: 3.1.2
- Gymnasium: 1.2.3

Model loaded ✅


#### 🧱 2. Ortamı Hazırlayın

Eğitilmiş modeli çalıştırmadan önce yeni bir bölüm başlatmak için ortamı yükleyin ve sıfırlayın.

In [30]:
env = gym.make("CliffWalking-v1", render_mode="human")
env = DummyVecEnv([lambda: env])

obs = env.reset()  # VecEnv -> sadece obs döner (info yok)
print("Env reset ✅ | obs shape/type:", type(obs), getattr(obs, "shape", None))

Env reset ✅ | obs shape/type: <class 'numpy.ndarray'> (1,)


#### 3. 🧠 En İyi Sonraki Eylemi Tahmin Edin

Mevcut gözleme dayalı olarak sonraki eylemi seçmek için modelin `.predict()` metodunu kullanın.

In [31]:
action, state = model.predict(obs, deterministic=True)
print("Predicted action:", action)

Predicted action: [0]


#### 4. 🚶‍♂️ Adımı Atın

- Tahmin edilen eylemi uygulamak için `.step(action)` kullanın.  
- Bu şunları döndürür:
  - Yeni durum  
  - Ödül  
  - Bir `done` bayrağı (bölüm bitmiş ise)  
  - Ek bilgi (varsa)

Ajanınız garip hareketler yaparsa veya sıkışırsa, paniklemeyın: muhtemelen yeterince uzun eğitim görmemiştir. Daha uzun eğitmeyi denemeden önce meydan okumanın sonuna kadar gidin 😉

In [32]:
obs, rewards, dones, infos = env.step(action)

print("New obs:", obs)
print("Reward:", rewards)
print("Done:", dones)
print("Info:", infos)

New obs: [24]
Reward: [-1.]
Done: [False]
Info: [{'prob': 1.0, 'TimeLimit.truncated': False}]


#### 5. 🖼️ Ortamı Görselleştirin

Eylem gerçekleştirildikten sonra ortamın mevcut durumunu görselleştirmek için `.render()` kullanın.

In [33]:
env.render()
print("Rendered ✅")

Rendered ✅


#### 6.  🧹 Ortamı Kapatın

Ortamı düzgün bir şekilde kapatmak ve sistem kaynaklarını serbest bırakmak için `.close()` kullanın.

In [34]:
env.close()
print("Env closed ✅")

Env closed ✅


---

### Bölüm 7: Tam Etkileşim Döngüsünü Uygulayın

Bu son alıştırmada, önceden eğitilmiş modelinizin bölüm bitene kadar ortamla adım adım etkileşim kurduğu tam bir döngü oluşturacaksınız.

- Eylemleri tekrarlamak için bir `while` döngüsü kullanın.
- Eylemleri seçmek için `.predict()` ve bunları uygulamak için `.step()` çağırın.
- Bir sonraki predict iterasyonuna yeni durumu beslediğinizden emin olun.
- Ne zaman döngüden çıkacağınızı bilmek için `.step()`'den gelen `done` bayrağını kullanın.
- Her adımdan sonra ajanı aksiyon halinde görselleştirmek için `.render()` çağırmayı unutmayın.

Yine, ajanınız garip hareketler seçerse veya sonsuz döngüye girerse, paniklemeyın: muhtemelen yeterince uzun eğitim görmemiştir. Ajanınız sonsuz döngüye girdiyse, hücrenin yürütülmesini durdurun.

👉 Daha uzun eğitmeyi denemeden önce meydan okumanın sonuna kadar gidin.

In [35]:
import gymnasium as gym
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv
import time

In [36]:
model_path = "dqn_cliffwalking_fast"
model = DQN.load(model_path, print_system_info=True)
print("Model loaded ✅")

== CURRENT SYSTEM INFO ==
- OS: macOS-26.2-arm64-arm-64bit Darwin Kernel Version 25.2.0: Tue Nov 18 21:09:56 PST 2025; root:xnu-12377.61.12~1/RELEASE_ARM64_T6041
- Python: 3.12.9
- Stable-Baselines3: 2.7.1
- PyTorch: 2.9.1
- GPU Enabled: False
- Numpy: 1.26.4
- Cloudpickle: 3.1.2
- Gymnasium: 1.2.3

== SAVED MODEL SYSTEM INFO ==
- OS: macOS-26.2-arm64-arm-64bit Darwin Kernel Version 25.2.0: Tue Nov 18 21:09:56 PST 2025; root:xnu-12377.61.12~1/RELEASE_ARM64_T6041
- Python: 3.12.9
- Stable-Baselines3: 2.7.1
- PyTorch: 2.9.1
- GPU Enabled: False
- Numpy: 1.26.4
- Cloudpickle: 3.1.2
- Gymnasium: 1.2.3

Model loaded ✅


In [37]:
env = gym.make("CliffWalking-v1", render_mode="human")
env = DummyVecEnv([lambda: env])

obs = env.reset()
print("Episode started ✅ | initial obs:", obs)

Episode started ✅ | initial obs: [36]


In [38]:
max_steps = 300   # güvenlik bariyeri
step_count = 0
total_reward = 0.0

done = False

while not done and step_count < max_steps:
    action, _ = model.predict(obs, deterministic=True)

    obs, rewards, dones, infos = env.step(action)

    # VecEnv olduğu için rewards/dones array; tek env var -> 0. eleman
    r = float(rewards[0])
    d = bool(dones[0])

    total_reward += r
    step_count += 1
    done = d

    env.render()

    # İstersen adımları ekrana dök (çok spam olmasın diye kısa tuttum)
    print(f"step={step_count:03d} | action={int(action[0])} | reward={r} | done={done}")

    # animasyon daha anlaşılır olsun diye (istersen kapat)
    time.sleep(0.05)

print("\nEpisode finished ✅")
print("Total steps:", step_count)
print("Total reward:", total_reward)
print("Last info:", infos[0])

step=001 | action=0 | reward=-1.0 | done=False
step=002 | action=1 | reward=-1.0 | done=False
step=003 | action=3 | reward=-1.0 | done=False
step=004 | action=1 | reward=-1.0 | done=False
step=005 | action=3 | reward=-1.0 | done=False
step=006 | action=1 | reward=-1.0 | done=False
step=007 | action=3 | reward=-1.0 | done=False
step=008 | action=1 | reward=-1.0 | done=False
step=009 | action=3 | reward=-1.0 | done=False
step=010 | action=1 | reward=-1.0 | done=False
step=011 | action=3 | reward=-1.0 | done=False
step=012 | action=1 | reward=-1.0 | done=False
step=013 | action=3 | reward=-1.0 | done=False
step=014 | action=1 | reward=-1.0 | done=False
step=015 | action=3 | reward=-1.0 | done=False
step=016 | action=1 | reward=-1.0 | done=False
step=017 | action=3 | reward=-1.0 | done=False
step=018 | action=1 | reward=-1.0 | done=False
step=019 | action=3 | reward=-1.0 | done=False
step=020 | action=1 | reward=-1.0 | done=False
step=021 | action=3 | reward=-1.0 | done=False
step=022 | ac

In [39]:
env.close()
print("Env closed ✅")

Env closed ✅


### 🤔 Ajanınızın Davranışından Memnun Değil misiniz?

Endişelenmeyin — seçenekleriniz var:

- 🏋️‍♂️ **Daha uzun eğitin** → Performansı artırmak için bölüm sayısını artırmayı deneyin.
- 📦 **Veya bizim önceden eğitilmiş modelimizi yükleyin** →  
  **500.000 bölüm** için bir tane eğittik — indirmek için aşağıdaki hücrenin yorumunu kaldırın ve çalıştırın.

In [40]:
!curl https://d37p7d5kaxknzw.cloudfront.net/projects/best_dqn_cliffwalking.zip > best_dqn_cliffwalking.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  139k  100  139k    0     0   168k      0 --:--:-- --:--:-- --:--:--  168k


In [41]:
from stable_baselines3 import DQN

best_model = DQN.load("best_dqn_cliffwalking.zip", print_system_info=True)
print("Best model loaded ✅")

== CURRENT SYSTEM INFO ==
- OS: macOS-26.2-arm64-arm-64bit Darwin Kernel Version 25.2.0: Tue Nov 18 21:09:56 PST 2025; root:xnu-12377.61.12~1/RELEASE_ARM64_T6041
- Python: 3.12.9
- Stable-Baselines3: 2.7.1
- PyTorch: 2.9.1
- GPU Enabled: False
- Numpy: 1.26.4
- Cloudpickle: 3.1.2
- Gymnasium: 1.2.3

== SAVED MODEL SYSTEM INFO ==
- OS: macOS-14.1-x86_64-i386-64bit Darwin Kernel Version 23.1.0: Mon Oct  9 21:27:27 PDT 2023; root:xnu-10002.41.9~6/RELEASE_X86_64
- Python: 3.12.9
- Stable-Baselines3: 2.4.1
- PyTorch: 2.2.2
- GPU Enabled: False
- Numpy: 1.26.4
- Cloudpickle: 3.1.1
- Gymnasium: 1.0.0

Best model loaded ✅


In [42]:
action, _ = best_model.predict(obs, deterministic=True)

### ❄️ Son not: `is_slippery=True` Anlamak

Dokümantasyonu okurken `is_slippery` parametresinin ne yaptığını merak etmiş olabilirsiniz.

`CliffWalking`'de `is_slippery=True` ayarlamak, ajanın eylemlerine rastgelelik ekler — ajanın kayıp istenmeyen bir yöne hareket etme şansını tanıtır. `is_slippery=True` ile eğitim yapmak, belirsizlikle başa çıkabilen ajanlar oluşturur — gerçek dünya uygulamaları için kritik bir beceri.

#### 🎯 Neden stokastisite ekleyelim?

- **🌍 Gerçekçilik** → Gerçek dünya ortamlarında bulunan belirsizliği simüle eder.  
- **🛡️ Sağlamlık** → Ajanların daha güvenilir, uyum sağlayabilen stratejiler öğrenmesine yardımcı olur.  
- **🔥 Meydan okuma** → Görevi daha zor ve çözülmesi daha ilginç hale getirir.

#### 🤔 Peki neden burada `True` yapmadık?

DQN algoritması basit kullanım durumları için tasarlanmıştır. `slippery=True`'nun `CliffWalking` ortamına eklediği yüksek stokastisite ile mücadele eder. Şunu düşünün: bu basit ortamda, amaçlanan yönü takip etmemek hemen tamamen farklı bir yöne gitmek anlamına gelir: 90 derece, hatta 180 derece! Basit bir algoritmanın öğrenmesi çok zor olurdu.

### Projeyi tamamladığınız için tebrikler 🏁